In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_probability as tfp
from helper_functions2 import get_imgs, load_caps_img, create_image_gen
import numpy as np
from tensorflow_probability import distributions

In [ ]:



comparison = tf.stack([test_data, images], axis=1)
b_sz, h, w, c = tf.shape(test_data)

# flip width and height
comparison = tf.transpose(comparison, perm=[0, 1, 3, 2, 4])
comparison = tf.reshape(comparison, [b_sz, 2 * w, h, c])

# reset width and height order
comparison = tf.transpose(comparison, perm=[0, 2, 1, 3])

In [8]:
def load_coco(use_all_data, image_shape, batch_size):
    dataset_name = 'train'
    train_capspath, train_imgspath = get_imgs(dataset_name)
    train_caps, train_imgs = load_caps_img(train_capspath, train_imgspath, dataset_name, use_all_data)
    
    print(set(train_imgs))
    
    dataset_name = 'val'
    val_capspath, val_imgspath = get_imgs(dataset_name)
    val_caps, val_imgs = load_caps_img(val_capspath, val_imgspath, dataset_name, use_all_data)
    
    dataset_name = 'test'
    test_capspath, test_imgspath = get_imgs(dataset_name)
    test_imgs = load_caps_img(test_capspath, test_imgspath, dataset_name, use_all_data)

    train_dataset = create_image_gen(set(train_imgs), batch_size, image_shape)
    val_dataset = create_image_gen(val_imgs, batch_size, image_shape)
    test_dataset = create_image_gen(test_imgs, batch_size, image_shape)
    
    return train_dataset, val_dataset, test_dataset

image_shape = 128; use_all_data=False
train_data_single, val_data_single, test_data_single = load_coco(use_all_data, image_shape, image_shape)

/users/apokkunu/trial/img/annotations/captions_train2014.json /users/apokkunu/trial/img/train2014/

dataset size: 82783

temp dataset size: 100
{'/users/apokkunu/trial/img/train2014/COCO_train2014_000000448139.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000515206.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000363847.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000032093.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000105844.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000083931.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000407369.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000238713.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000005396.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000265330.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000321059.jpg', '/users/apokkunu/trial/img/train2014/COCO_train2014_000000581873.jpg', '/u

In [3]:
strategy = tf.distribute.MirroredStrategy()
print ('\nNumber of devices: {}'.format(strategy.num_replicas_in_sync), flush=True)

GLOBAL_BATCH_SIZE = image_shape * strategy.num_replicas_in_sync

train_data = strategy.experimental_distribute_dataset(train_data_single)
val_data = strategy.experimental_distribute_dataset(val_data_single)
test_data = strategy.experimental_distribute_dataset(test_data_single)

n_list = list(train_data_single.as_numpy_iterator())
batches_per_epoch = len(n_list)

sample_batch = next(train_data_single.as_numpy_iterator())

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)

Number of devices: 1


In [4]:
def int_shape(x):
    return list(map(int, x.get_shape()))

def log_sum_exp(x):
    """ numerically stable log_sum_exp implementation that prevents overflow """
    axis = len(x.get_shape())-1
    m = tf.reduce_max(x, axis)
    m2 = tf.reduce_max(x, axis, keepdims=True)
    return m + tf.math.log(tf.reduce_sum(tf.exp(x-m2), axis))

def log_prob_from_logits(x):
    """ numerically stable log_softmax implementation that prevents overflow """
    axis = len(x.get_shape())-1
    m = tf.reduce_max(x, axis, keepdims=True)
    return x - m - tf.math.log(tf.reduce_sum(tf.math.exp(x-m), axis, keepdims=True))

def discretized_mix_logistic_loss(x, l, sum_all=True):
    """ log-likelihood for mixture of discretized logistics, assumes the data has been rescaled to [-1,1] interval """
    xs = int_shape(x) # true image (i.e. labels) to regress to, e.g. (B,32,32,3)
    ls = int_shape(l) # predicted distribution, e.g. (B,32,32,100)
    nr_mix = int(ls[-1] / 10) # here and below: unpacking the params of the mixture of logistics
    logit_probs = l[:, :, :, :nr_mix]
    
    print(l[:,:,:,nr_mix:].shape, xs + [nr_mix*3])
    
    l = tf.reshape(l[:,:,:,nr_mix:], xs + [nr_mix*3])
    means = l[:,:,:,:,:nr_mix]
    log_scales = tf.maximum(l[:,:,:,:,nr_mix:2*nr_mix], -7.)
    coeffs = tf.nn.tanh(l[:,:,:,:,2*nr_mix:3*nr_mix])
    x = tf.reshape(x, xs + [1]) + tf.zeros(xs + [nr_mix]) # here and below: getting the means and adjusting them based on preceding sub-pixels
    m2 = tf.reshape(means[:,:,:,1,:] + coeffs[:, :, :, 0, :] * x[:, :, :, 0, :], [xs[0],xs[1],xs[2],1,nr_mix])
    m3 = tf.reshape(means[:, :, :, 2, :] + coeffs[:, :, :, 1, :] * x[:, :, :, 0, :] + coeffs[:, :, :, 2, :] * x[:, :, :, 1, :], [xs[0],xs[1],xs[2],1,nr_mix])
    means = tf.concat([tf.reshape(means[:,:,:,0,:], [xs[0],xs[1],xs[2],1,nr_mix]), m2, m3],3)
    
    centered_x = x - means
    inv_stdv = tf.exp(-log_scales)
    plus_in = inv_stdv * (centered_x + 1./255.)
    cdf_plus = tf.nn.sigmoid(plus_in)
    min_in = inv_stdv * (centered_x - 1./255.)
    cdf_min = tf.nn.sigmoid(min_in)
    log_cdf_plus = plus_in - tf.nn.softplus(plus_in) # log probability for edge case of 0 (before scaling)
    log_one_minus_cdf_min = -tf.nn.softplus(min_in) # log probability for edge case of 255 (before scaling)
    cdf_delta = cdf_plus - cdf_min # probability for all other cases
    mid_in = inv_stdv * centered_x
    log_pdf_mid = mid_in - log_scales - 2.*tf.nn.softplus(mid_in) # log probability in the center of the bin, to be used in extreme cases (not actually used in our code)

    # now select the right output: left edge case, right edge case, normal case, extremely low prob case (doesn't actually happen for us)

    # this is what we are really doing, but using the robust version below for extreme cases in other applications and to avoid NaN issue with tf.select()
    # log_probs = tf.select(x < -0.999, log_cdf_plus, tf.select(x > 0.999, log_one_minus_cdf_min, tf.math.log(cdf_delta)))
    
    # robust version, that still works if probabilities are below 1e-5 (which never happens in our code)
    # tensorflow backpropagates through tf.select() by multiplying with zero instead of selecting: this requires use to use some ugly tricks to avoid potential NaNs
    # the 1e-12 in tf.maximum(cdf_delta, 1e-12) is never actually used as output, it's purely there to get around the tf.select() gradient issue
    # if the probability on a sub-pixel is below 1e-5, we use an approximation based on the assumption that the log-density is constant in the bin of the observed sub-pixel value
    log_probs = tf.where(x < -0.999, log_cdf_plus, tf.where(x > 0.999, log_one_minus_cdf_min, 
                                                            tf.where(cdf_delta > 1e-5, tf.math.log(tf.maximum(cdf_delta, 1e-12)), log_pdf_mid - np.log(127.5))))

    log_probs = tf.reduce_sum(log_probs,3) + log_prob_from_logits(logit_probs)
    if sum_all:
        return -tf.reduce_sum(log_sum_exp(log_probs))
    else:
        return -tf.reduce_sum(log_sum_exp(log_probs),[1,2])

def sample_from_discretized_mix_logistic(l, nr_mix):
    ls = int_shape(l)
    xs = ls[:-1] + [3]
    
    # unpack parameters
    logit_probs = l[:, :, :, :nr_mix]
    l = tf.reshape(l[:, :, :, nr_mix:], xs + [nr_mix*3])
    
    # sample mixture indicator from softmax
    sel = tf.one_hot(tf.argmax(logit_probs - tf.math.log(-tf.math.log(tf.random.uniform(logit_probs.get_shape(), minval=1e-5, maxval=1. - 1e-5))), 3), 
                     depth=nr_mix, dtype=tf.float32)
    sel = tf.reshape(sel, xs[:-1] + [1,nr_mix])
    
    # select logistic parameters
    means = tf.reduce_sum(l[:,:,:,:,:nr_mix]*sel,4)
    log_scales = tf.maximum(tf.reduce_sum(l[:,:,:,:,nr_mix:2*nr_mix]*sel,4), -7.)
    coeffs = tf.reduce_sum(tf.nn.tanh(l[:,:,:,:,2*nr_mix:3*nr_mix])*sel,4)
    
    # sample from logistic & clip to interval
    # we don't actually round to the nearest 8bit value when sampling
    u = tf.random.uniform(means.get_shape(), minval=1e-5, maxval=1. - 1e-5)
    x = means + tf.exp(log_scales)*(tf.math.log(u) - tf.math.log(1. - u))
    x0 = tf.minimum(tf.maximum(x[:,:,:,0], -1.), 1.)
    x1 = tf.minimum(tf.maximum(x[:,:,:,1] + coeffs[:,:,:,0]*x0, -1.), 1.)
    x2 = tf.minimum(tf.maximum(x[:,:,:,2] + coeffs[:,:,:,1]*x0 + coeffs[:,:,:,2]*x1, -1.), 1.)
    return tf.concat([tf.reshape(x0,xs[:-1]+[1]), tf.reshape(x1,xs[:-1]+[1]), tf.reshape(x2,xs[:-1]+[1])],3)

In [19]:
l = tf.random.normal([3, 64, 64, 100])
ls = int_shape(l)
xs = ls[:-1] + [3]
print(ls, xs)

nr_mix = 10

logit_probs = l[:, :, :, :nr_mix]
fff = l[:, :, :, nr_mix:]
print(fff.shape)

print(xs + [nr_mix*3])

l = tf.reshape(fff, xs + [nr_mix*3])
print(logit_probs.shape, l.shape)

[3, 64, 64, 100] [3, 64, 64, 3]
(3, 64, 64, 90)
[3, 64, 64, 3, 30]
(3, 64, 64, 10) (3, 64, 64, 3, 30)


In [8]:
l = tf.random.normal([3, 64, 64, 60])
ls = int_shape(l)
print(ls)

nr_mix = int(ls[-1] / 10)
print(nr_mix)

xs = ls[:-1] + [3]
print(xs, xs + [nr_mix*3], 'xs')

# unpack parameters
logit_probs = l[:, :, :, :nr_mix]
print(logit_probs.shape)

ff = l[:, :, :, nr_mix:]
print(ff.shape, 'ff')

l = tf.reshape(ff, xs + [nr_mix*3])

[3, 64, 64, 60]
6
[3, 64, 64, 3] [3, 64, 64, 3, 18] xs
(3, 64, 64, 6)
(3, 64, 64, 54) ff


In [32]:
l = tf.random.normal([3, 128, 128, 40])
ls = int_shape(l) # predicted distribution, e.g. (B,32,32,100)
print(ls)
nr_mix = int(ls[-1] / 10)
print(nr_mix)
a = sample_from_discretized_mix_logistic(l, nr_mix)
print(a.shape)

[3, 128, 128, 40]
4
(3, 128, 128, 3)


In [7]:
x = tf.random.normal([100, 64, 64, 3])
l = tf.random.normal([100, 64, 64, 60])
a = discretized_mix_logistic_loss(x, l)
print(a)

(100, 64, 64, 54) [100, 64, 64, 3, 18]
tf.Tensor(6250638.5, shape=(), dtype=float32)


In [6]:
reconstruction = tf.random.normal([3, 128, 128, 3])
inputs = tf.random.normal([3, 128, 128, 3]) * 2

log_probs = distributions.Bernoulli(logits=reconstruction, dtype=tf.float32, allow_nan_stats=False).log_prob(inputs)
print(log_probs.shape)

(3, 128, 128, 3)


In [30]:
reconstruction = tf.expand_dims(tf.random.normal([3, 128, 128, 3]), axis=4)
distributions.Categorical(logits=reconstruction, dtype=tf.float32, allow_nan_stats=False)

<tfp.distributions.Categorical 'Categorical' batch_shape=[3, 128, 128, 3] event_shape=[] dtype=float32>

In [14]:
[distributions.Normal(loc=0., scale=1.0)]*3

[<tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>,
 <tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>,
 <tfp.distributions.Normal 'Normal' batch_shape=[] event_shape=[] dtype=float32>]

In [11]:
tf.tile(tf.ones([1, 1]), [3, 1])

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>

In [13]:
reconstruction = tf.expand_dims(tf.random.normal([3, 128, 128, 3]), axis=4)
inputs = tf.expand_dims(tf.random.normal([3, 128, 128, 3]), axis=4) * 2

log_probs = distributions.Mixture(distributions.Categorical(logits=reconstruction, dtype=tf.float32, allow_nan_stats=False), 
                                                 components=[distributions.Normal(loc=0., scale=1.0)], validate_args=True, allow_nan_stats=False).log_prob(inputs)
print(log_probs.shape)

ValueError: components[0] batch shape must be compatible with cat shape and other component batch shapes

In [ ]:
def load_mnist(batch_size, binary=True):
    train_ds, test_ds = tfds.load("mnist", split=["train", "test"], shuffle_files=True, batch_size=batch_size, as_supervised=True)

    def transform(image, label):
        image = tf.image.resize_with_crop_or_pad(image, 32, 32)
        image = tf.cast(image, dtype=tf.float32)
        if binary:
            image = tfp.distributions.Bernoulli(probs=image, dtype=tf.float32).sample()
        else:
            image /= 255
        return image, label

    return train_ds.map(transform), test_ds.map(transform)

train_data, test_ds = load_mnist(128)

In [ ]:
# def load_mnist(batch_size, binary=True):
#     train_ds, test_ds = tfds.load("mnist", split=["train", "test"], shuffle_files=True, batch_size=batch_size, as_supervised=True)
    
#     def transform(image, label):
#         image = tf.image.resize_with_crop_or_pad(image, 32, 32)
#         image = tf.cast(image, dtype=tf.float32)
#         if binary:
#             image = tfp.distributions.Bernoulli(probs=image, dtype=tf.float32).sample()
#         else:
#             image /= 255
#         return image, label
    
#     def extract_raw(tf_ds):
#         d = []
#         for i in tf_ds:
#             i = i[0]
#             for j in i.numpy():
#                 d.append(j)
#         return np.array(d)
    
#     train_ds = train_ds.map(transform)
#     test_ds = test_ds.map(transform)
    
#     x_train = extract_raw(train_ds)
#     x_test = extract_raw(test_ds)
#     return x_train, x_test

# batch_size = 128
# train_data, test_ds = load_mnist(batch_size)

In [ ]:
# sample_batch, lb = next(train_data.as_numpy_iterator())

In [ ]:
sample_batch, lb = max(enumerate(train_data.as_numpy_iterator()))[1]

sample_batch = next(train_data.as_numpy_iterator())

In [ ]:
sample_batch.shape

In [ ]:
def load_coco(use_all_data, image_shape, batch_size):
    dataset_name = 'train'
    train_capspath, train_imgspath = get_imgs(dataset_name)
    train_caps, train_imgs = load_caps_img(train_capspath, train_imgspath, dataset_name, use_all_data)
    
    dataset_name = 'val'
    val_capspath, val_imgspath = get_imgs(dataset_name)
    val_caps, val_imgs = load_caps_img(val_capspath, val_imgspath, dataset_name, use_all_data)

    dataset_name = 'test'
    test_capspath, test_imgspath = get_imgs(dataset_name)
    test_imgs = load_caps_img(test_capspath, test_imgspath, dataset_name, use_all_data)

    train_dataset = create_image_gen(train_imgs, batch_size, image_shape)
    val_dataset = create_image_gen(val_imgs, batch_size, image_shape)
    test_dataset = create_image_gen(test_imgs, batch_size, image_shape)
    
    return train_dataset, val_dataset, test_dataset

use_all_data = True
image_shape = 128
batch_size = 128
train_data, val_dataset, test_data = load_coco(use_all_data, image_shape, batch_size)

In [ ]:
len(list(train_data.as_numpy_iterator()))